**ÉTAPE 0** : préparation des données

In [81]:
import process 
import pandas as pd

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

There are no NaN values in the dataframe


**ÉTAPE 1** : Phase d'entraînement

1. Obtention de la matrice de corrélation des actifs sur une fenêtre arrière de 30 jours (1 mois)

In [82]:
L = 365
W = process.training_phase(lookback_window=L, df_cleaned=df_cleaned, number_of_clusters=20)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:46: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:47: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [83]:
import numpy as np

def portfolio_return(evaluation_window):
    evaluation_set = df_cleaned.iloc[:, L+1:L+1+evaluation_window]

    evaluation_set = df_cleaned.iloc[:, L+1:L+1+evaluation_window]

    portfolio_return = pd.DataFrame(index=evaluation_set.columns, columns=['portfolio return'], data=np.zeros(len(evaluation_set.columns)))

    for elem1 in portfolio_return.index:
        for elem2 in W:
            portfolio_return.loc[str(elem1), 'portfolio return'] += elem2[1]*evaluation_set.loc[elem2[0], str(elem1)]

In [84]:
def Sharpe_and_PnL(portfolio_return):
    portfolio_return = np.array(portfolio_return)

    # Calcul du rendement moyen du portefeuille
    Rp = np.mean(portfolio_return)

    # Calcul de l'écart type du portefeuille
    sigma_p = np.std(portfolio_return)

    # Taux sans risque (ou rendement moyen du marché)
    Rf = 0.02  # Remplacez par le taux sans risque ou le rendement moyen du marché approprié

    # Calcul du Sharpe ratio
    SR = (Rp - Rf) / sigma_p

    # Calcul des PNL
    PNL = np.cumsum(portfolio_return)

    return SR, PNL

In [85]:
import plotly.graph_objects as go


def plot_PnL(SR, PNL):

    # Création du graphique
    fig = go.Figure()

    # Ajout de la trace pour les PNL
    fig.add_trace(go.Scatter(x=np.arange(31, 61), y=PNL, mode='lines', name='PNL'))

    # Ajout d'une ligne pour le Sharpe ratio
    fig.add_shape(
        type='line',
        x0=31, x1=60, y0=0, y1=0,
        line=dict(color='red', width=2),
        opacity=0.7,
        name='Sharpe Ratio'
    )

    # Ajout d'une annotation pour le Sharpe ratio
    fig.add_annotation(
        x=45,
        y=PNL[-1],
        text=f'Sharpe Ratio: {SR:.4f}',
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor='red',
        font=dict(size=10),
        bordercolor='red',
        borderwidth=2,
        bgcolor='white',
        opacity=0.8
    )

    # Mise en forme du graphique
    fig.update_layout(
        title='Évolution des PNL et Sharpe Ratio',
        xaxis_title='Période',
        yaxis_title='PNL Cumulatif',
        legend=dict(x=0, y=1),
        template='plotly_white'
    )

    # Affichage du graphique
    fig.show()

In [80]:

# Création du tableau synthétique
data = {
    'Rendement Moyen': [Rp],
    'Écart Type': [sigma_p],
    'Taux sans Risque': [Rf],
    'Sharpe Ratio': [SR],
    'PNL Finale': [PNL[-1]]
}

table = pd.DataFrame(data)

# Affichage du tableau
print(table)

   Rendement Moyen  Écart Type  Taux sans Risque  Sharpe Ratio  PNL Finale
0         0.016567    0.156385              0.02     -0.021955    0.496999
